<a href="https://colab.research.google.com/github/jyellow/Hands-On-Large-Language-Models/blob/main/chapter07/Chapter%207%20-%20Advanced%20Text%20Generation%20Techniques%20and%20Tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Chapter 7 - Advanced Text Generation Techniques and Tools</h1>
<i>Going beyond prompt engineering.</i>

<a href="https://www.amazon.com/Hands-Large-Language-Models-Understanding/dp/1098150961"><img src="https://img.shields.io/badge/Buy%20the%20Book!-grey?logo=amazon"></a>
<a href="https://www.oreilly.com/library/view/hands-on-large-language/9781098150952/"><img src="https://img.shields.io/badge/O'Reilly-white.svg?logo=data:image/svg%2bxml;base64,PHN2ZyB3aWR0aD0iMzQiIGhlaWdodD0iMjciIHZpZXdCb3g9IjAgMCAzNCAyNyIgZmlsbD0ibm9uZSIgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzIwMDAvc3ZnIj4KPGNpcmNsZSBjeD0iMTMiIGN5PSIxNCIgcj0iMTEiIHN0cm9rZT0iI0Q0MDEwMSIgc3Ryb2tlLXdpZHRoPSI0Ii8+CjxjaXJjbGUgY3g9IjMwLjUiIGN5PSIzLjUiIHI9IjMuNSIgZmlsbD0iI0Q0MDEwMSIvPgo8L3N2Zz4K"></a>
<a href="https://github.com/HandsOnLLM/Hands-On-Large-Language-Models"><img src="https://img.shields.io/badge/GitHub%20Repository-black?logo=github"></a>
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/HandsOnLLM/Hands-On-Large-Language-Models/blob/main/chapter07/Chapter%207%20-%20Advanced%20Text%20Generation%20Techniques%20and%20Tools.ipynb)

---

This notebook is for Chapter 7 of the [Hands-On Large Language Models](https://www.amazon.com/Hands-Large-Language-Models-Understanding/dp/1098150961) book by [Jay Alammar](https://www.linkedin.com/in/jalammar) and [Maarten Grootendorst](https://www.linkedin.com/in/mgrootendorst/).

---

<a href="https://www.amazon.com/Hands-Large-Language-Models-Understanding/dp/1098150961">
<img src="https://raw.githubusercontent.com/HandsOnLLM/Hands-On-Large-Language-Models/main/images/book_cover.png" width="350"/></a>

### [OPTIONAL] - Installing Packages on <img src="https://colab.google/static/images/icons/colab.png" width=100>

If you are viewing this notebook on Google Colab (or any other cloud vendor), you need to **uncomment and run** the following codeblock to install the dependencies for this chapter:

---

💡 **NOTE**: We will want to use a GPU to run the examples in this notebook. In Google Colab, go to
**Runtime > Change runtime type > Hardware accelerator > GPU > GPU type > T4**.

---


In [1]:
# %%capture
!pip install -q langchain>=0.1.17 openai>=1.13.3 langchain_openai>=0.1.6 transformers>=4.40.1 datasets>=2.18.0 accelerate>=0.27.2 sentence-transformers>=2.5.1 duckduckgo-search>=5.2.2 langchain_community
!pip install -q --no-cache-dir llama-cpp-python==0.2.90 --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu122

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://abetlen.github.io/llama-cpp-python/whl/cu122
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.8/443.8 MB 285.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 179.0 MB/s eta 0:00:00


# Loading an LLM

In [2]:
!wget https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf

# If this command does not work for you, you can use the link directly to download the model
# https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf

--2024-12-03 03:42:28--  https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf
Resolving huggingface.co (huggingface.co)... 13.35.210.114, 13.35.210.77, 13.35.210.66, ...
Connecting to huggingface.co (huggingface.co)|13.35.210.114|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/41/c8/41c860f65b01de5dc4c68b00d84cead799d3e7c48e38ee749f4c6057776e2e9e/5d99003e395775659b0dde3f941d88ff378b2837a8dc3a2ea94222ab1420fad3?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27Phi-3-mini-4k-instruct-fp16.gguf%3B+filename%3D%22Phi-3-mini-4k-instruct-fp16.gguf%22%3B&Expires=1733456548&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczMzQ1NjU0OH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzQxL2M4LzQxYzg2MGY2NWIwMWRlNWRjNGM2OGIwMGQ4NGNlYWQ3OTlkM2U3YzQ4ZTM4ZWU3NDlmNGM2MDU3Nzc2ZTJlOWUvNWQ5OTAwM2UzOTU3NzU2NTliMGRkZTNmOTQxZDg

In [3]:
from langchain import LlamaCpp

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="Phi-3-mini-4k-instruct-fp16.gguf",
    n_gpu_layers=-1,
    max_tokens=500,
    n_ctx=2048,
    seed=42,
    verbose=False
)

In [4]:
llm.invoke("Hi! My name is Maarten. What is 1 + 1?")

''

### Chains

In [5]:
from langchain import PromptTemplate

# Create a prompt template with the "input_prompt" variable
template = """<s><|user|>
{input_prompt}<|end|>
<|assistant|>"""
prompt = PromptTemplate(
    template=template,
    input_variables=["input_prompt"]
)

In [6]:
basic_chain = prompt | llm

In [7]:
# Use the chain
basic_chain.invoke(
    {
        "input_prompt": "Hi! My name is Maarten. What is 1 + 1?",
    }
)

/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1138: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


" Hello, Maarten! The answer to 1 + 1 is 2. It's a basic arithmetic operation where you add one unit to another unit.\n\nHere is the calculation:\n\n1 + 1 = 2"

### Multiple Chains

In [8]:
from langchain import LLMChain

# Create a chain for the title of our story
template = """<s><|user|>
Create a title for a story about {summary}. Only return the title.<|end|>
<|assistant|>"""
title_prompt = PromptTemplate(template=template, input_variables=["summary"])
title = LLMChain(llm=llm, prompt=title_prompt, output_key="title")

<ipython-input-8-61dd782c6da9>:8: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  title = LLMChain(llm=llm, prompt=title_prompt, output_key="title")


In [9]:
title.invoke({"summary": "a girl that lost her mother"})

/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1138: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'summary': 'a girl that lost her mother',
 'title': ' "Whispers of Solace: A Tale of Grief and Redemption"'}

In [10]:
# Create a chain for the character description using the summary and title
template = """<s><|user|>
Describe the main character of a story about {summary} with the title {title}. Use only two sentences.<|end|>
<|assistant|>"""
character_prompt = PromptTemplate(
    template=template, input_variables=["summary", "title"]
)
character = LLMChain(llm=llm, prompt=character_prompt, output_key="character")

In [11]:
# Create a chain for the story using the summary, title, and character description
template = """<s><|user|>
Create a story about {summary} with the title {title}. The main charachter is: {character}. Only return the story and it cannot be longer than one paragraph<|end|>
<|assistant|>"""
story_prompt = PromptTemplate(
    template=template, input_variables=["summary", "title", "character"]
)
story = LLMChain(llm=llm, prompt=story_prompt, output_key="story")

In [12]:
# Combine all three components to create the full chain
llm_chain = title | character | story

In [13]:
llm_chain.invoke("a girl that lost her mother")

/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1138: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1138: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1138: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'summary': 'a girl that lost her mother',
 'title': ' "Echoes of Love: A Girl\'s Journey Through Grief"',
 'character': ' The protagonist, Lily, is a resilient and compassionate young girl who has been deeply affected by the loss of her mother but finds solace in nature as she embarks on a transformative journey to find healing and acceptance. Her strength and vulnerability intertwine as she learns to navigate grief while honoring the enduring love shared with her mother through cherished memories and newfound purpose.',
 'story': ' "Echoes of Love: A Girl\'s Journey Through Grief" tells the poignant tale of Lily, a resilient young girl whose world shattered when her mother passed away. Consumed by grief but not defeated by sorrow, she embarked on an introspective journey through nature\'s serene landscapes and bustling city streets alike. Amidst the rustle of leaves and whispers of wind, Lily discovered solace, finding echoes of her mother\'s love in every sunrise and moonlit night. 

# Memory

In [14]:
# Let's give the LLM our name
basic_chain.invoke({"input_prompt": "Hi! My name is Maarten. What is 1 + 1?"})

/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1138: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


" Hello Maarten! The answer to 1 + 1 is 2. It's a basic arithmetic calculation."

In [15]:
# Next, we ask the LLM to reproduce the name
basic_chain.invoke({"input_prompt": "What is my name?"})

/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1138: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


" I'm unable to determine your name as I don't have access to that information. If you provide me with context or details, I might be able to assist you further!\n\nHowever, if this is a general inquiry about how one could find out their own name in a practical sense, the answer involves checking any personal identification documents like a passport, driver's license, or birth certificate. Always remember that as an AI, I am designed to respect user privacy and confidentiality."

## ConversationBuffer

In [16]:
# Create an updated prompt template to include a chat history
template = """<s><|user|>Current conversation:{chat_history}

{input_prompt}<|end|>
<|assistant|>"""

prompt = PromptTemplate(
    template=template,
    input_variables=["input_prompt", "chat_history"]
)

In [17]:
from langchain.memory import ConversationBufferMemory

# Define the type of Memory we will use
memory = ConversationBufferMemory(memory_key="chat_history")

# Chain the LLM, Prompt, and Memory together
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

<ipython-input-17-9823403f20ac>:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history")


In [18]:
# Generate a conversation and ask a basic question
llm_chain.invoke({"input_prompt": "Hi! My name is Maarten. What is 1 + 1?"})

/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1138: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'Hi! My name is Maarten. What is 1 + 1?',
 'chat_history': '',
 'text': " The answer to your question, Maarten, is that 1 + 1 equals 2. It's a basic arithmetic operation where you combine two units together."}

In [19]:
# Does the LLM remember the name we gave it?
llm_chain.invoke({"input_prompt": "What is my name?"})

/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1138: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'What is my name?',
 'chat_history': "Human: Hi! My name is Maarten. What is 1 + 1?\nAI:  The answer to your question, Maarten, is that 1 + 1 equals 2. It's a basic arithmetic operation where you combine two units together.",
 'text': ' Your name is Maarten. I am an AI and do not have a personal name, but you can refer to me as the AI assistant.'}

## ConversationBufferMemoryWindow

In [20]:
from langchain.memory import ConversationBufferWindowMemory

# Retain only the last 2 conversations in memory
memory = ConversationBufferWindowMemory(k=2, memory_key="chat_history")

# Chain the LLM, Prompt, and Memory together
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

<ipython-input-20-046ef635f261>:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=2, memory_key="chat_history")


In [21]:
# Ask two questions and generate two conversations in its memory
llm_chain.invoke({"input_prompt":"Hi! My name is Maarten and I am 33 years old. What is 1 + 1?"})
llm_chain.invoke({"input_prompt":"What is 3 + 3?"})

/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1138: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1138: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'What is 3 + 3?',
 'chat_history': "Human: Hi! My name is Maarten and I am 33 years old. What is 1 + 1?\nAI:  The sum of 1 + 1 is 2. This question, although simple, serves as a pleasant introduction to our conversation. It's always great meeting you, Maarten!\nHere we can also take this opportunity to learn more about your interests or experiences if you wish to share them. How has your day been so far?",
 'text': " The sum of 3 + 3 is 6. This question keeps the friendly tone going and encourages further conversation, such as asking how you've been today or what activities you've enjoyed recently!\n\nSo in total, we have:\n1 + 1 = 2\n3 + 3 = 6\nIt's a great way to build rapport and connect on basic topics before potentially delving into more personal subjects."}

In [22]:
# Check whether it knows the name we gave it
llm_chain.invoke({"input_prompt":"What is my name?"})

/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1138: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'What is my name?',
 'chat_history': "Human: Hi! My name is Maarten and I am 33 years old. What is 1 + 1?\nAI:  The sum of 1 + 1 is 2. This question, although simple, serves as a pleasant introduction to our conversation. It's always great meeting you, Maarten!\nHere we can also take this opportunity to learn more about your interests or experiences if you wish to share them. How has your day been so far?\nHuman: What is 3 + 3?\nAI:  The sum of 3 + 3 is 6. This question keeps the friendly tone going and encourages further conversation, such as asking how you've been today or what activities you've enjoyed recently!\n\nSo in total, we have:\n1 + 1 = 2\n3 + 3 = 6\nIt's a great way to build rapport and connect on basic topics before potentially delving into more personal subjects.",
 'text': " You are calling myself your AI assistant! As for your name, it's nice to meet you again, Maarten. If there's anything specific you'd like to discuss or ask about today, I'm here to 

In [23]:
# Check whether it knows the age we gave it
llm_chain.invoke({"input_prompt":"What is my age?"})

/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1138: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'What is my age?',
 'chat_history': "Human: What is 3 + 3?\nAI:  The sum of 3 + 3 is 6. This question keeps the friendly tone going and encourages further conversation, such as asking how you've been today or what activities you've enjoyed recently!\n\nSo in total, we have:\n1 + 1 = 2\n3 + 3 = 6\nIt's a great way to build rapport and connect on basic topics before potentially delving into more personal subjects.\nHuman: What is my name?\nAI:  You are calling myself your AI assistant! As for your name, it's nice to meet you again, Maarten. If there's anything specific you'd like to discuss or ask about today, I'm here to help. However, if we were to stick with our fun math-related topic: 3 + 3 indeed equals 6!",
 'text': " As an AI, I don't have the capability to know personal information about individuals unless it has been shared during our conversation. This ensures your privacy and confidentiality. If you need assistance with something related to general knowledge o

## ConversationSummary

In [24]:
# Create a summary prompt template
summary_prompt_template = """<s><|user|>Summarize the conversations and update with the new lines.

Current summary:
{summary}

new lines of conversation:
{new_lines}

New summary:<|end|>
<|assistant|>"""
summary_prompt = PromptTemplate(
    input_variables=["new_lines", "summary"],
    template=summary_prompt_template
)

In [25]:
from langchain.memory import ConversationSummaryMemory

# Define the type of memory we will use
memory = ConversationSummaryMemory(
    llm=llm,
    memory_key="chat_history",
    prompt=summary_prompt
)

# Chain the LLM, prompt, and memory together
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

<ipython-input-25-905e44fca19a>:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(


In [26]:
# Generate a conversation and ask for the name
llm_chain.invoke({"input_prompt": "Hi! My name is Maarten. What is 1 + 1?"})
llm_chain.invoke({"input_prompt": "What is my name?"})

/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1138: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1138: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1138: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1138: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'What is my name?',
 'chat_history': ' Maarten introduces himself and asks the AI about the sum of 1+1. The AI responds by stating that the answer is 2, explaining it as a fundamental mathematical operation.',
 'text': ' Your name was not mentioned in this conversation; you referred to yourself as "User." Maarten introduced himself when he started the conversation.'}

In [27]:
# Check whether it has summarized everything thus far
llm_chain.invoke({"input_prompt": "What was the first question I asked?"})

/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1138: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1138: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'What was the first question I asked?',
 'chat_history': ' In the conversation, Maarten introduces himself and inquires about the sum of 1+1 from the AI. The AI confirms that the result is 2, explaining it\'s a basic mathematical operation. When asked about his name, the AI clarifies that no name was mentioned in this discussion; the user identified themselves as "User." Maarten initiated the conversation and referred to himself by name when he introduced himself.',
 'text': ' The first question you asked was: "Can you tell me what 1+1 is?"'}

In [28]:
# Check what the summary is thus far
memory.load_memory_variables({})

{'chat_history': ' In the conversation, Maarten introduces himself and asks about the sum of 1+1 from the AI. The AI confirms that the result is 2, explaining it\'s a basic mathematical operation. When asked for his name, the AI clarifies that no name was mentioned in this discussion; the user identified themselves as "User." Maarten initiated the conversation and referred to himself by name when he introduced himself. The human then inquires about the first question they asked which was: "Can you tell me what 1+1 is?"'}

# Agents

In [29]:
import os
from langchain_openai import ChatOpenAI
from google.colab import userdata

openai_llm = ChatOpenAI(
    model_name="deepseek-chat",
    temperature=0,
    openai_api_base="https://api.deepseek.com/v1",
    openai_api_key=userdata.get('DS_API_KEY')
)

In [30]:
# Create the ReAct template
react_template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

prompt = PromptTemplate(
    template=react_template,
    input_variables=["tools", "tool_names", "input", "agent_scratchpad"]
)

In [31]:
from langchain.agents import load_tools, Tool
from langchain.tools import DuckDuckGoSearchResults

# You can create the tool to pass to an agent
search = DuckDuckGoSearchResults()
search_tool = Tool(
    name="duckduck",
    description="A web search engine. Use this to as a search engine for general queries.",
    func=search.run,
)

# Prepare tools
tools = load_tools(["llm-math"], llm=openai_llm)
tools.append(search_tool)

In [32]:
from langchain.agents import AgentExecutor, create_react_agent

# Construct the ReAct agent
agent = create_react_agent(openai_llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [33]:
# What is the Price of a MacBook Pro?
agent_executor.invoke(
    {
        "input": "What is the current price of a MacBook Pro in USD? How much would it cost in EUR if the exchange rate is 0.85 EUR for 1 USD?"
    }
)



> Entering new AgentExecutor chain...
Thought: I need to find the current price of a MacBook Pro in USD first. Then, I can convert that price to EUR using the given exchange rate.

Action: duckduck
Action Input: "current price of a MacBook Pro in USD"
snippet: The 14-inch MacBook Pro M4 is one of the best MacBooks we've tested and comes with our highest recommendation, especially at this lower price. MacBook Pro 14" (M4) was $1,599 now $1,399 @ Amazon, title: The new M4 MacBook Pro 14-inch is already $200 off before Black Friday ..., link: https://www.tomsguide.com/computing/macbooks/hurry-the-new-m4-macbook-pro-14-inch-just-dropped-to-lowest-price-yet-at-amazon, snippet: As of April 2024, the latest MacBook Pro 14-inch M3 model started with a retail price of 1,599 U.S. dollars while the 16-inch M3 Pro version began with a retail price of 2,499 U.S. dollars., title: Apple MacBook Pro price comparison 2024 - Statista, link: https://www.statista.com/statistics/1360426/apple-macbook-pro

{'input': 'What is the current price of a MacBook Pro in USD? How much would it cost in EUR if the exchange rate is 0.85 EUR for 1 USD?',
 'output': 'The current price of a MacBook Pro 14-inch M4 model is $1,399 in USD. Converting this to EUR at the exchange rate of 0.85 EUR for 1 USD, the price would be approximately €1,189.15.'}